In [1]:
import gradio as gr
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
model_path = "bert-book-classifier"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [3]:
df = pd.read_csv("book_clean.csv")


In [4]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

df["simple_categories"] = df["categories"].map(category_mapping)

In [5]:
df = df[df["simple_categories"].isin(["Fiction", "Nonfiction"])].dropna(subset=["description"])

In [6]:
def classify_description(desc):
    inputs = tokenizer(desc, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return "Fiction" if pred == 1 else "Nonfiction"

In [7]:
def recommend_books(description, category_choice):
    if category_choice == "All":
        predicted_category = classify_description(description)
    else:
        predicted_category = category_choice

    filtered_books = df[df['simple_categories'].str.lower() == predicted_category.lower()].copy()

    # Fallback for missing thumbnails
    filtered_books['thumbnail'] = filtered_books['thumbnail'].apply(
        lambda x: x if isinstance(x, str) and x.strip() else "cover-not-found.jpg"
    )

    # Fallback for missing titles
    filtered_books['title'] = filtered_books['title'].fillna("Untitled")

    # Prepare gallery and book list
    gallery_data = [(row['thumbnail'], row['title']) for _, row in filtered_books.iterrows()]
    book_list = filtered_books.to_dict(orient="records")

    return predicted_category, gallery_data, book_list


In [8]:
def show_details(evt: gr.SelectData, book_list):
    book = book_list[evt.index]
    return gr.Image.update(value=book["thumbnail"]), gr.Text.update(value=f"{book['bookname']}\n\n{book['desc']}")


In [12]:
with gr.Blocks() as demo:
    gr.Markdown("##  🪶QuillAI : Book Recommender")

    with gr.Row():
        description = gr.Textbox(label="Enter a book description")
        category = gr.Dropdown(["All", "Fiction", "Nonfiction"], value="All", label="Select a category")

    predict_btn = gr.Button("Find recommendations")
    predicted_category = gr.Textbox(label="Predicted Category", interactive=False)

    gallery = gr.Gallery(label="Recommendations", columns=4)
    

    book_state = gr.State()

    predict_btn.click(fn=recommend_books, inputs=[description, category], outputs=[predicted_category, gallery, book_state])
    gallery.select(fn=show_details, inputs=[book_state], outputs=[book_image, book_info])

In [13]:
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://03b9be91ecd029d836.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
